# Rag Deployment Development

This is a quick example how we can easily deploy a rag service in code.


In [6]:
# Use the Model Library to pull the 
from model_library.client import ModelLibraryClient
model_library_url = "http://127.0.0.1:8002"
model_library_api_key = "adam"
library = ModelLibraryClient(model_library_url, model_library_api_key)

In [7]:
library.get_all()

{'detail': 'Invalid API Key'}

In [5]:
# Todo streamline this
rag_template = {t["id"]:t for t in library.get_all()["ModelDeploymentTemplateCard"]}["rag-tool"]

KeyError: 'ModelDeploymentTemplateCard'

In [45]:

from model_library.models import *
namespace = "adam2"
API_key = "XXX"

# Create a User
user = UserInformation(id=namespace, namespace=namespace, API_key=API_key)

# Create a Config
config = library.create_kubernetes_deployment(
    template=rag_template,
    user=user.dict()
)

<Response [200]>


In [ ]:
# Deploy Model using a Client
#kube_watcher_url = 
#kube_watcher_api_key = 
#kube_watcher = KubeWatcherClient(kube_watcher_url, kube_watcher_api_key)
#res = kube_watcher.deploy_generic_model({"config":st.session_state.config})

In [46]:
# Use the library 
from kube_watcher.kube_core import KubeAPI
kube_watcher = KubeAPI(in_cluster=False)
kube_watcher.deploy_generic_model(config)

{'successful': ["[[{'api_version': 'v1',\n 'kind': 'Namespace',\n 'metadata': {'annotations': None,\n              'creation_timestamp': datetime.datetime(2024, 3, 4, 10, 56, 13, tzinfo=tzutc()),\n              'deletion_grace_period_seconds': None,\n              'deletion_timestamp': None,\n              'finalizers': None,\n              'generate_name': None,\n              'generation': None,\n              'labels': {'kubernetes.io/metadata.name': 'adam2'},\n              'managed_fields': [{'api_version': 'v1',\n                                  'fields_type': 'FieldsV1',\n                                  'fields_v1': {'f:metadata': {'f:labels': {'.': {},\n                                                                            'f:kubernetes.io/metadata.name': {}}}},\n                                  'manager': 'OpenAPI-Generator',\n                                  'operation': 'Update',\n                                  'subresource': None,\n                             

In [37]:
kube_watcher. 

{'successful': [],
 'failed': [{'yaml': 'apiVersion: v1\nkind: Namespace\nmetadata:\n  name: adam\n',
   'error': 'Error from server (Conflict): {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"namespaces \\"adam\\" already exists","reason":"AlreadyExists","details":{"name":"adam","kind":"namespaces"},"code":409}\n'},
  {'yaml': '\napiVersion: apps/v1\nkind: Deployment\nmetadata:\n  namespace: adam\n  name: rag-tool-test\nspec:\n  replicas: 1\n  selector:\n    matchLabels:\n      app: rag-tool-test\n  template:\n    metadata:\n      labels:\n        app: rag-tool-test\n    spec:\n      containers:\n      - name: rag-tool-container\n        image: python:3.8-slim\n        ports:\n        - containerPort: 8000\n        env:\n        - name: RAG_USE_AUTH\n          value: "False"\n        - name: RAG_MASTER_KEY\n          value: ""\n        command: ["/bin/sh", "-c"]\n        args:\n        - |\n          apt-get update && apt-get install -y git\n          gi